In [79]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jeju-dataset/sample_submission.csv
/kaggle/input/jeju-dataset/train.csv
/kaggle/input/jeju-dataset/test.csv
/kaggle/input/jeju-dataset/international_trade.csv


In [80]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

DATA_PATH = '/kaggle/input/jeju-dataset/'

SEED = 42

train = pd.read_csv(f'{DATA_PATH}train.csv')
test = pd.read_csv(f'{DATA_PATH}test.csv')
submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')
trade = pd.read_csv(f'{DATA_PATH}international_trade.csv')


In [81]:
train['name'] = train['ID'].apply(lambda x: x[:6])
test['name'] = test['ID'].apply(lambda x: x[:6])

In [82]:
train.rename(columns={'supply(kg)': 'supply_kg'}, inplace=True)
train.rename(columns={'price(원/kg)': 'price'}, inplace=True)


# Feature

## Time Feature

In [83]:
# 'timestamp' 칼럼을 datetime 형식으로 변환
train['timestamp'] = pd.to_datetime(train['timestamp'])

# 연, 월, 일, 요일, 연중 일, 연중 주간 추출 및 칼럼 추가
train['year'] = train['timestamp'].dt.year
train['month'] = train['timestamp'].dt.month
train['day'] = train['timestamp'].dt.day
# train['week'] = train['timestamp'].dt.week

train['day_of_week'] = train['timestamp'].dt.dayofweek  # 월요일(0)부터 일요일(6)까지
train['day_of_year'] = train['timestamp'].dt.dayofyear  # 연중 일
train['week_of_year'] = train['timestamp'].dt.isocalendar().week  # 연중 주간 (ISO 주차 기준)

# 주말 여부 칼럼 추가 (0: 주중, 1: 주말)
train['is_weekend'] = train['timestamp'].dt.weekday // 5

# 계절 칼럼 추가 (1: 봄, 2: 여름, 3: 가을, 4: 겨울)
def get_season(month):
    if 3 <= month <= 5:
        return 1  # 봄
    elif 6 <= month <= 8:
        return 2  # 여름
    elif 9 <= month <= 11:
        return 3  # 가을
    else:
        return 4  # 겨울

train['season'] = train['month'].apply(get_season)

train.head(5)

,ID,timestamp,item,corporation,location,supply_kg,price,name,year,month,day,day_of_week,day_of_year,week_of_year,is_weekend,season
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,TG_A_J,2019,1,1,1,1,1,0,4
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,TG_A_J,2019,1,2,2,2,1,0,4
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,TG_A_J,2019,1,3,3,3,1,0,4
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,TG_A_J,2019,1,4,4,4,1,0,4
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,TG_A_J,2019,1,5,5,5,1,1,4


In [84]:
# 'timestamp' 칼럼을 datetime 형식으로 변환
test['timestamp'] = pd.to_datetime(test['timestamp'])

# 연, 월, 일, 요일, 연중 일, 연중 주간 추출 및 칼럼 추가
test['year'] = test['timestamp'].dt.year
test['month'] = test['timestamp'].dt.month
test['day'] = test['timestamp'].dt.day
# test['week'] = test['timestamp'].dt.week

test['day_of_week'] = test['timestamp'].dt.dayofweek  # 월요일(0)부터 일요일(6)까지
test['day_of_year'] = test['timestamp'].dt.dayofyear  # 연중 일
test['week_of_year'] = test['timestamp'].dt.isocalendar().week  # 연중 주간 (ISO 주차 기준)

# 주말 여부 칼럼 추가 (0: 주중, 1: 주말)
test['is_weekend'] = test['timestamp'].dt.weekday // 5

# 계절 칼럼 추가 (1: 봄, 2: 여름, 3: 가을, 4: 겨울)
def get_season(month):
    if 3 <= month <= 5:
        return 1  # 봄
    elif 6 <= month <= 8:
        return 2  # 여름
    elif 9 <= month <= 11:
        return 3  # 가을
    else:
        return 4  # 겨울

test['season'] = test['month'].apply(get_season)

test.head(5)

,ID,timestamp,item,corporation,location,name,year,month,day,day_of_week,day_of_year,week_of_year,is_weekend,season
0,TG_A_J_20230304,2023-03-04,TG,A,J,TG_A_J,2023,3,4,5,63,9,1,1
1,TG_A_J_20230305,2023-03-05,TG,A,J,TG_A_J,2023,3,5,6,64,9,1,1
2,TG_A_J_20230306,2023-03-06,TG,A,J,TG_A_J,2023,3,6,0,65,10,0,1
3,TG_A_J_20230307,2023-03-07,TG,A,J,TG_A_J,2023,3,7,1,66,10,0,1
4,TG_A_J_20230308,2023-03-08,TG,A,J,TG_A_J,2023,3,8,2,67,10,0,1


## Price Feature

In [85]:
# name_weekly_avg_price = train.groupby(['name', 'week'])['price'].mean().reset_index()
# name_weekly_avg_price  #2067 데이터
name_monthly_avg_price = train.groupby(['name', 'month'])['price'].mean().reset_index()
name_monthly_avg_price #468데이터
name_yearly_avg_price = train.groupby(['name', 'year'])['price'].mean().reset_index()
name_yearly_avg_price #195데이터
name_season_avg_price = train.groupby(['name', 'season'])['price'].mean().reset_index()
name_season_avg_price.shape #156 데이터

(156, 3)

In [86]:
name_day_of_week_avg_price = train.groupby(['name', 'day_of_week'])['price'].mean().reset_index()
name_day_of_week_avg_price.shape #273 데이터
name_day_of_year_avg_price = train.groupby(['name', 'day_of_year'])['price'].mean().reset_index()
name_day_of_year_avg_price.shape #14274 데이터
name_week_of_year_avg_price = train.groupby(['name', 'week_of_year'])['price'].mean().reset_index()
name_week_of_year_avg_price.shape #2067 데이터

(2067, 3)

In [87]:
# 주간 평균 가격을 추가
# train = train.merge(name_weekly_avg_price, on=['name', 'week'], how='left', suffixes=('', '_weekly_avg_price'))

# 월간 평균 가격을 추가
train = train.merge(name_monthly_avg_price, on=['name', 'month'], how='left', suffixes=('', '_monthly_avg_price'))

# 연간 평균 가격을 추가
train = train.merge(name_yearly_avg_price, on=['name', 'year'], how='left', suffixes=('', '_yearly_avg_price'))

# 계절별 평균 가격을 추가
train = train.merge(name_season_avg_price, on=['name', 'season'], how='left', suffixes=('', '_season_avg_price'))

# 요일별 평균 가격을 추가
train = train.merge(name_day_of_week_avg_price, on=['name', 'day_of_week'], how='left', suffixes=('', '_day_of_week_avg_price'))

# 연중 일자별 평균 가격을 추가
train = train.merge(name_day_of_year_avg_price, on=['name', 'day_of_year'], how='left', suffixes=('', '_day_of_year_avg_price'))

# 연중 주차별 평균 가격을 추가
train = train.merge(name_week_of_year_avg_price, on=['name', 'week_of_year'], how='left', suffixes=('', '_week_of_year_avg_price'))


In [88]:
# 주간 평균 가격을 추가
# test = test.merge(name_weekly_avg_price, on=['name', 'week'], how='left', suffixes=('', '_weekly_avg_price'))

# 월간 평균 가격을 추가
test = test.merge(name_monthly_avg_price, on=['name', 'month'], how='left', suffixes=('', '_monthly_avg_price'))

# 연간 평균 가격을 추가
test = test.merge(name_yearly_avg_price, on=['name', 'year'], how='left', suffixes=('', '_yearly_avg_price'))

# 계절별 평균 가격을 추가
test = test.merge(name_season_avg_price, on=['name', 'season'], how='left', suffixes=('', '_season_avg_price'))

# 요일별 평균 가격을 추가
test = test.merge(name_day_of_week_avg_price, on=['name', 'day_of_week'], how='left', suffixes=('', '_day_of_week_avg_price'))

# 연중 일자별 평균 가격을 추가
test = test.merge(name_day_of_year_avg_price, on=['name', 'day_of_year'], how='left', suffixes=('', '_day_of_year_avg_price'))

# 연중 주차별 평균 가격을 추가
test = test.merge(name_week_of_year_avg_price, on=['name', 'week_of_year'], how='left', suffixes=('', '_week_of_year_avg_price'))


## Supply Feature

In [89]:
# 'name' 별로 그룹화하여 그룹별 평균 가격 계산
# name_weekly_avg_supply = train.groupby(['name', 'week'])['supply_kg'].mean().reset_index()
# name_weekly_avg_supply  #2067 데이터
name_monthly_avg_supply = train.groupby(['name', 'month'])['supply_kg'].mean().reset_index()
name_monthly_avg_supply #468데이터
name_yearly_avg_supply = train.groupby(['name', 'year'])['supply_kg'].mean().reset_index()
name_yearly_avg_supply #195데이터
name_season_avg_supply = train.groupby(['name', 'season'])['supply_kg'].mean().reset_index()
name_season_avg_supply.shape #156 데이터

(156, 3)

In [90]:
name_day_of_week_avg_supply = train.groupby(['name', 'day_of_week'])['supply_kg'].mean().reset_index()
name_day_of_week_avg_supply.shape #273 데이터
name_day_of_year_avg_supply = train.groupby(['name', 'day_of_year'])['supply_kg'].mean().reset_index()
name_day_of_year_avg_supply.shape #14274 데이터
name_week_of_year_avg_supply = train.groupby(['name', 'week_of_year'])['supply_kg'].mean().reset_index()
name_week_of_year_avg_supply.shape #2067 데이터


(2067, 3)

In [91]:
# 주간 평균 유통물량을 추가
# train = train.merge(name_weekly_avg_supply, on=['name', 'week'], how='left', suffixes=('', '_weekly_avg_supply'))

# 월간 평균 유통물량을 추가
train = train.merge(name_monthly_avg_supply, on=['name', 'month'], how='left', suffixes=('', '_monthly_avg_supply'))

# 연간 평균 유통물량을 추가
train = train.merge(name_yearly_avg_supply, on=['name', 'year'], how='left', suffixes=('', '_yearly_avg_supply'))

# 계절별 평균 유통물량을 추가
train = train.merge(name_season_avg_supply, on=['name', 'season'], how='left', suffixes=('', '_season_avg_supply'))

# 요일별 평균 유통물량을 추가
train = train.merge(name_day_of_week_avg_supply, on=['name', 'day_of_week'], how='left', suffixes=('', '_day_of_week_avg_supply'))

# 연중 일자별 평균 유통물량을 추가
train = train.merge(name_day_of_year_avg_supply, on=['name', 'day_of_year'], how='left', suffixes=('', '_day_of_year_avg_supply'))

# 연중 주차별 평균 유통물량을 추가
train = train.merge(name_week_of_year_avg_supply, on=['name', 'week_of_year'], how='left', suffixes=('', '_week_of_year_avg_supply'))


In [92]:
# 주간 평균 유통물량을 추가
# test = test.merge(name_weekly_avg_supply, on=['name', 'week'], how='left', suffixes=('', '_weekly_avg_supply'))

# 월간 평균 유통물량을 추가
test = test.merge(name_monthly_avg_supply, on=['name', 'month'], how='left', suffixes=('', '_monthly_avg_supply'))

# 연간 평균 유통물량을 추가
test = test.merge(name_yearly_avg_supply, on=['name', 'year'], how='left', suffixes=('', '_yearly_avg_supply'))

# 계절별 평균 유통물량을 추가
test = test.merge(name_season_avg_supply, on=['name', 'season'], how='left', suffixes=('', '_season_avg_supply'))

# 요일별 평균 유통물량을 추가
test = test.merge(name_day_of_week_avg_supply, on=['name', 'day_of_week'], how='left', suffixes=('', '_day_of_week_avg_supply'))

# 연중 일자별 평균 유통물량을 추가
test = test.merge(name_day_of_year_avg_supply, on=['name', 'day_of_year'], how='left', suffixes=('', '_day_of_year_avg_supply'))

# 연중 주차별 평균 유통물량을 추가
test = test.merge(name_week_of_year_avg_supply, on=['name', 'week_of_year'], how='left', suffixes=('', '_week_of_year_avg_supply'))


In [93]:
train.isnull().sum()

ID                                   0
timestamp                            0
item                                 0
corporation                          0
location                             0
supply_kg                            0
price                                0
name                                 0
year                                 0
month                                0
day                                  0
day_of_week                          0
day_of_year                          0
week_of_year                         0
is_weekend                           0
season                               0
price_monthly_avg_price              0
price_yearly_avg_price               0
price_season_avg_price               0
price_day_of_week_avg_price          0
price_day_of_year_avg_price          0
price_week_of_year_avg_price         0
supply_kg_monthly_avg_supply         0
supply_kg_yearly_avg_supply          0
supply_kg_season_avg_supply          0
supply_kg_day_of_week_avg

## Trade Feature

In [94]:
trade.columns = ['time','item','export_kg','export_price','import_kg','import_price','trade_balance']
trade

,time,item,export_kg,export_price,import_kg,import_price,trade_balance
0,2019-01,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990
1,2019-01,양파,821330,222,4003206,1118,-896
2,2019-01,쪽파,60,1,93405,128,-127
3,2019-01,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562
4,2019-01,방울다다기 양배추,0,0,7580,38,-38
...,...,...,...,...,...,...,...
1269,2023-02,포포(papaw)[파파야(papaya)],0,0,23830,71,-71
1270,2023-02,사과,135165,351,0,0,351
1271,2023-02,배,2206012,5411,1,0,5411
1272,2023-02,신 체리[프루너스 체라서스(Prunus cerasus)],5,0,0,0,0


In [95]:
trade['time'] = pd.to_datetime(trade['time'])

trade['year'] = trade['time'].dt.year
trade['month'] = trade['time'].dt.month

In [96]:
mask = trade['item'].str.contains('감귤')
tg_index = trade[mask][['item','year','month','export_kg','export_price','import_kg','import_price','trade_balance']]
tg_index['item'] = tg_index['item'].replace('감귤','TG')
tg_index

mask = trade['item'].str.contains('브로콜리')
bc_index = trade[mask][['item','year','month','export_kg','export_price','import_kg','import_price','trade_balance']]
bc_index['item'] = bc_index['item'].replace('꽃양배추와 브로콜리(broccoli)','BC')
bc_index

mask = trade['item'].str.contains('순무')
rd_index = trade[mask][['item','year','month','export_kg','export_price','import_kg','import_price','trade_balance']]
rd_index['item'] = rd_index['item'].replace('순무','RD')
rd_index

mask = trade['item'].str.contains('당근')
cr_index = trade[mask][['item','year','month','export_kg','export_price','import_kg','import_price','trade_balance']]
cr_index['item'] = cr_index['item'].replace('당근','CR')
cr_index

mask = trade['item'].isin(['양배추'])
cb_index = trade[mask][['item','year','month','export_kg','export_price','import_kg','import_price','trade_balance']]
cb_index['item'] = cb_index['item'].str.replace('양배추', 'CB', regex=True)
cb_index

,item,year,month,export_kg,export_price,import_kg,import_price,trade_balance
5,CB,2019,1,184650,94,395802,90,4
30,CB,2019,2,182636,69,336142,77,-8
53,CB,2019,3,854925,354,356514,94,260
77,CB,2019,4,6494616,2658,343463,105,2553
101,CB,2019,5,1461639,662,270972,95,567
128,CB,2019,6,3286366,1656,271444,69,1586
155,CB,2019,7,1676629,798,472041,119,679
182,CB,2019,8,2465625,1154,280140,73,1081
209,CB,2019,9,1187087,566,226329,60,507
237,CB,2019,10,201297,105,635187,151,-46


In [97]:
merged_indices = pd.concat([tg_index, bc_index, rd_index, cr_index, cb_index], axis=0).reset_index().drop(columns='index')
merged_indices

item_monthly_avg = merged_indices.groupby(['item', 'month'], as_index=False).mean()
item_monthly_avg = item_monthly_avg.drop(columns = 'year')
item_monthly_avg

,item,month,export_kg,export_price,import_kg,import_price,trade_balance
0,BC,1,46.40,0.40,682774.20,751.80,-751.20
1,BC,2,423.20,1.00,403745.20,432.80,-431.80
2,BC,3,45.50,1.00,753128.50,819.50,-818.50
3,BC,4,33.50,0.25,890748.00,1106.00,-1105.75
4,BC,5,18.00,0.25,722724.25,856.25,-855.75
5,BC,6,12.00,0.00,513319.50,569.00,-569.00
6,BC,7,20.00,0.00,642013.25,744.00,-744.00
7,BC,8,8.25,0.00,1224899.75,1466.75,-1466.75
8,BC,9,6.00,0.00,1407729.00,1677.00,-1677.00
9,BC,10,6.00,0.00,971622.50,1081.00,-1081.00


In [98]:
train = pd.merge(train, item_monthly_avg, on=['item', 'month'], how='left')
test = pd.merge(test, item_monthly_avg, on=['item', 'month'], how='left')


## 범주형 Feature 처리

In [99]:
cat_cols = ['item','corporation','location']
train = pd.get_dummies(train, columns=cat_cols)
test = pd.get_dummies(test, columns=cat_cols)


In [100]:
train.columns

Index(['ID', 'timestamp', 'supply_kg', 'price', 'name', 'year', 'month', 'day',
       'day_of_week', 'day_of_year', 'week_of_year', 'is_weekend', 'season',
       'price_monthly_avg_price', 'price_yearly_avg_price',
       'price_season_avg_price', 'price_day_of_week_avg_price',
       'price_day_of_year_avg_price', 'price_week_of_year_avg_price',
       'supply_kg_monthly_avg_supply', 'supply_kg_yearly_avg_supply',
       'supply_kg_season_avg_supply', 'supply_kg_day_of_week_avg_supply',
       'supply_kg_day_of_year_avg_supply', 'supply_kg_week_of_year_avg_supply',
       'export_kg', 'export_price', 'import_kg', 'import_price',
       'trade_balance', 'item_BC', 'item_CB', 'item_CR', 'item_RD', 'item_TG',
       'corporation_A', 'corporation_B', 'corporation_C', 'corporation_D',
       'corporation_E', 'corporation_F', 'location_J', 'location_S'],
      dtype='object')

In [101]:
test.columns

Index(['ID', 'timestamp', 'name', 'year', 'month', 'day', 'day_of_week',
       'day_of_year', 'week_of_year', 'is_weekend', 'season', 'price',
       'price_yearly_avg_price', 'price_season_avg_price',
       'price_day_of_week_avg_price', 'price_day_of_year_avg_price',
       'price_week_of_year_avg_price', 'supply_kg',
       'supply_kg_yearly_avg_supply', 'supply_kg_season_avg_supply',
       'supply_kg_day_of_week_avg_supply', 'supply_kg_day_of_year_avg_supply',
       'supply_kg_week_of_year_avg_supply', 'export_kg', 'export_price',
       'import_kg', 'import_price', 'trade_balance', 'item_BC', 'item_CB',
       'item_CR', 'item_RD', 'item_TG', 'corporation_A', 'corporation_B',
       'corporation_C', 'corporation_D', 'corporation_E', 'corporation_F',
       'location_J', 'location_S'],
      dtype='object')

In [102]:
cols = ['year', 'month', 'day', 'day_of_week',
       'day_of_year', 'week_of_year', 'is_weekend', 'season',
        'price_monthly_avg_price',
       'price_yearly_avg_price', 'price_season_avg_price',
       'price_day_of_week_avg_price', 'price_day_of_year_avg_price',
       'price_week_of_year_avg_price', 
       'supply_kg_monthly_avg_supply', 'supply_kg_yearly_avg_supply',
       'supply_kg_season_avg_supply', 'supply_kg_day_of_week_avg_supply',
       'supply_kg_day_of_year_avg_supply', 'supply_kg_week_of_year_avg_supply',
       'export_kg', 'export_price', 'import_kg', 'import_price',
       'trade_balance', 'item_BC', 'item_CB', 'item_CR', 'item_RD', 'item_TG',
       'corporation_A', 'corporation_B', 'corporation_C', 'corporation_D',
       'corporation_E', 'corporation_F', 'location_J', 'location_S']

In [103]:
test = test.rename(columns = {'price':'price_monthly_avg_price','supply_kg':'supply_kg_monthly_avg_supply'})
test.columns

Index(['ID', 'timestamp', 'name', 'year', 'month', 'day', 'day_of_week',
       'day_of_year', 'week_of_year', 'is_weekend', 'season',
       'price_monthly_avg_price', 'price_yearly_avg_price',
       'price_season_avg_price', 'price_day_of_week_avg_price',
       'price_day_of_year_avg_price', 'price_week_of_year_avg_price',
       'supply_kg_monthly_avg_supply', 'supply_kg_yearly_avg_supply',
       'supply_kg_season_avg_supply', 'supply_kg_day_of_week_avg_supply',
       'supply_kg_day_of_year_avg_supply', 'supply_kg_week_of_year_avg_supply',
       'export_kg', 'export_price', 'import_kg', 'import_price',
       'trade_balance', 'item_BC', 'item_CB', 'item_CR', 'item_RD', 'item_TG',
       'corporation_A', 'corporation_B', 'corporation_C', 'corporation_D',
       'corporation_E', 'corporation_F', 'location_J', 'location_S'],
      dtype='object')

In [143]:
train.to_csv('tmp_train_feature_1107.csv',index=False)
test.to_csv('tmp_test_feature_1107.csv',index=False)

In [129]:
train = train.fillna(0)
test = test.fillna(0)

train_ft = train[cols].copy()
test_ft = test[cols].copy()

target = train['price']

In [130]:
# train.iloc[:,5:].corr()['price'].sort_values(ascending=False)

## Scaling

In [131]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit_transform(train_ft)

In [132]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit_transform(train_ft)
train_ft = scaler.fit_transform(train_ft)
test_ft = scaler.transform(test_ft)


In [133]:
train_ft

array([[-1.33333484, -1.51152084, -1.66797684, ..., -0.23249528,
         0.83405766, -0.83405766],
       [-1.33333484, -1.51152084, -1.55439691, ..., -0.23249528,
         0.83405766, -0.83405766],
       [-1.33333484, -1.51152084, -1.44081698, ..., -0.23249528,
         0.83405766, -0.83405766],
       ...,
       [ 1.99699556, -0.9433248 , -1.66797684, ...,  4.30116263,
         0.83405766, -0.83405766],
       [ 1.99699556, -0.9433248 , -1.55439691, ...,  4.30116263,
         0.83405766, -0.83405766],
       [ 1.99699556, -0.9433248 , -1.44081698, ...,  4.30116263,
         0.83405766, -0.83405766]])

# 전체 데이터

In [134]:
!pip install catboost

In [135]:
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold


In [136]:
from sklearn.metrics import mean_squared_error

def rmse(y_valid, pred):
    mse = mean_squared_error(y_valid, pred)
    return np.sqrt(mse)

In [137]:
from sklearn.metrics import make_scorer
scorer = make_scorer(rmse, greater_is_better=False)

In [138]:
from sklearn.ensemble import StackingRegressor

cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)


estimators = [
    ("rf", RandomForestRegressor(random_state=SEED)),
    ("lgbm",LGBMRegressor(random_state=SEED)),
    ("xgboost",XGBRegressor(random_state=SEED)),
    ("cat",CatBoostRegressor(random_state=SEED,iterations=200,verbose=0))
]

params = {
    "estimators" : estimators,
    "final_estimator" : LinearRegression(),
    "cv" : cv,
    "n_jobs":-1
}

model_ensemble= StackingRegressor(**params)

rmse_scores = cross_val_score(model_ensemble, train_ft, target, cv=cv, scoring=scorer, n_jobs=-1)
np.mean(np.abs(rmse_scores))

#Stacking (rf, lgbm, xgb, cat) : 755.4509535074882 (실제점수 : 797.10784)

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


153:	learn: 639.1971137	total: 9.49s	remaining: 2.83s
154:	learn: 638.7323624	total: 9.53s	remaining: 2.77s
155:	learn: 638.0673914	total: 9.57s	remaining: 2.7s
156:	learn: 637.7294467	total: 9.61s	remaining: 2.63s
157:	learn: 637.2924280	total: 9.66s	remaining: 2.57s
158:	learn: 636.6807159	total: 9.71s	remaining: 2.5s
159:	learn: 636.5041105	total: 9.76s	remaining: 2.44s
160:	learn: 636.1319145	total: 9.83s	remaining: 2.38s
161:	learn: 635.7566082	total: 9.89s	remaining: 2.32s
162:	learn: 634.3824890	total: 9.96s	remaining: 2.26s
163:	learn: 633.7281686	total: 10s	remaining: 2.2s
164:	learn: 632.9444491	total: 10s	remaining: 2.13s
165:	learn: 632.4643086	total: 10.1s	remaining: 2.08s
166:	learn: 631.8138080	total: 10.2s	remaining: 2.01s
167:	learn: 630.9534816	total: 10.2s	remaining: 1.95s
168:	learn: 630.4485367	total: 10.3s	remaining: 1.89s
169:	learn: 629.9955162	total: 10.4s	remaining: 1.83s
170:	learn: 629.7417238	total: 10.4s	remaining: 1.76s
171:	learn: 628.7527634	total: 10.5

/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


153:	learn: 635.7511314	total: 10.1s	remaining: 3.01s
154:	learn: 634.9936855	total: 10.1s	remaining: 2.94s
155:	learn: 634.6939360	total: 10.2s	remaining: 2.87s
156:	learn: 633.8874008	total: 10.2s	remaining: 2.8s
157:	learn: 633.6478453	total: 10.3s	remaining: 2.72s
158:	learn: 632.5958065	total: 10.3s	remaining: 2.65s
159:	learn: 631.9564879	total: 10.3s	remaining: 2.58s
160:	learn: 630.8863606	total: 10.4s	remaining: 2.51s
161:	learn: 630.2061397	total: 10.4s	remaining: 2.45s
162:	learn: 629.2795466	total: 10.5s	remaining: 2.39s
163:	learn: 628.5171938	total: 10.6s	remaining: 2.33s
164:	learn: 627.1164953	total: 10.7s	remaining: 2.27s
165:	learn: 625.9508058	total: 10.8s	remaining: 2.21s
166:	learn: 625.3913429	total: 10.9s	remaining: 2.16s
167:	learn: 625.0632847	total: 11s	remaining: 2.09s
168:	learn: 624.4680995	total: 11.1s	remaining: 2.03s
169:	learn: 623.5726636	total: 11.1s	remaining: 1.97s
170:	learn: 622.9349837	total: 11.2s	remaining: 1.9s
171:	learn: 621.3262162	total: 1

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


752.7679857992932

In [139]:
model = model_ensemble
model.fit(train_ft,target)

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


StackingRegressor(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                  estimators=[('rf', RandomForestRegressor(random_state=42)),
                              ('lgbm', LGBMRegressor(random_state=42)),
                              ('xgboost',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categ...
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=42, ...)),
                              ('cat',
                               <catboost.core.CatBoostRegressor object at 0x7fde18204520>)],
                  final_estimator=LinearRegression(), n_jobs=-1)

In [140]:
pred = model.predict(test_ft)
pred

array([3458.07326679,   92.16194149, 3202.81619346, ...,  504.64939386,
        394.71036838,  360.71263197])

In [141]:
submission['answer'] = pred

submission.to_csv('tmp_submission_total_1107.csv',index=False)

# 상품 (name) 별 

In [191]:
train = pd.read_csv('tmp_train_feature_1107.csv')
test = pd.read_csv('tmp_test_feature_1107.csv')

train.shape, test.shape

((59397, 43), (1092, 41))

In [192]:
train.columns

Index(['ID', 'timestamp', 'supply_kg', 'price', 'name', 'year', 'month', 'day',
       'day_of_week', 'day_of_year', 'week_of_year', 'is_weekend', 'season',
       'price_monthly_avg_price', 'price_yearly_avg_price',
       'price_season_avg_price', 'price_day_of_week_avg_price',
       'price_day_of_year_avg_price', 'price_week_of_year_avg_price',
       'supply_kg_monthly_avg_supply', 'supply_kg_yearly_avg_supply',
       'supply_kg_season_avg_supply', 'supply_kg_day_of_week_avg_supply',
       'supply_kg_day_of_year_avg_supply', 'supply_kg_week_of_year_avg_supply',
       'export_kg', 'export_price', 'import_kg', 'import_price',
       'trade_balance', 'item_BC', 'item_CB', 'item_CR', 'item_RD', 'item_TG',
       'corporation_A', 'corporation_B', 'corporation_C', 'corporation_D',
       'corporation_E', 'corporation_F', 'location_J', 'location_S'],
      dtype='object')

In [204]:
train_ft = train.copy()
test_ft = test.copy()

In [205]:
for i,name in enumerate(train_ft.name.unique()):
    globals()[f'train_ft_{i}'] = train_ft[train_ft['name'] == name]

for i,name in enumerate(test_ft.name.unique()):
    globals()[f'test_ft_{i}'] = test_ft[test_ft['name'] == name]


In [206]:
cols_train = ['price','year', 'month', 'day', 'day_of_week',
       'day_of_year', 'week_of_year', 'is_weekend', 'season',
        'price_monthly_avg_price',
       'price_yearly_avg_price', 'price_season_avg_price',
       'price_day_of_week_avg_price', 'price_day_of_year_avg_price',
       'price_week_of_year_avg_price', 
       'supply_kg_monthly_avg_supply', 'supply_kg_yearly_avg_supply',
       'supply_kg_season_avg_supply', 'supply_kg_day_of_week_avg_supply',
       'supply_kg_day_of_year_avg_supply', 'supply_kg_week_of_year_avg_supply',
       'export_kg', 'export_price', 'import_kg', 'import_price',
       'trade_balance', 'item_BC', 'item_CB', 'item_CR', 'item_RD', 'item_TG',
       'corporation_A', 'corporation_B', 'corporation_C', 'corporation_D',
       'corporation_E', 'corporation_F', 'location_J', 'location_S']

In [207]:
cols_test = [col for col in cols_train if col != 'price']
cols_test

['year',
 'month',
 'day',
 'day_of_week',
 'day_of_year',
 'week_of_year',
 'is_weekend',
 'season',
 'price_monthly_avg_price',
 'price_yearly_avg_price',
 'price_season_avg_price',
 'price_day_of_week_avg_price',
 'price_day_of_year_avg_price',
 'price_week_of_year_avg_price',
 'supply_kg_monthly_avg_supply',
 'supply_kg_yearly_avg_supply',
 'supply_kg_season_avg_supply',
 'supply_kg_day_of_week_avg_supply',
 'supply_kg_day_of_year_avg_supply',
 'supply_kg_week_of_year_avg_supply',
 'export_kg',
 'export_price',
 'import_kg',
 'import_price',
 'trade_balance',
 'item_BC',
 'item_CB',
 'item_CR',
 'item_RD',
 'item_TG',
 'corporation_A',
 'corporation_B',
 'corporation_C',
 'corporation_D',
 'corporation_E',
 'corporation_F',
 'location_J',
 'location_S']

In [208]:
for i in range(0, 39):
    globals()[f'train_ft_{i}'] = globals()[f'train_ft_{i}'][cols_train]
    globals()[f'test_ft_{i}'] = globals()[f'test_ft_{i}'][cols_test]

In [209]:
train_ft_0

,price,year,month,day,day_of_week,day_of_year,week_of_year,is_weekend,season,price_monthly_avg_price,...,item_RD,item_TG,corporation_A,corporation_B,corporation_C,corporation_D,corporation_E,corporation_F,location_J,location_S
0,0.0,2019,1,1,1,1,1,0,4,1403.858065,...,False,True,True,False,False,False,False,False,True,False
1,0.0,2019,1,2,2,2,1,0,4,1403.858065,...,False,True,True,False,False,False,False,False,True,False
2,1728.0,2019,1,3,3,3,1,0,4,1403.858065,...,False,True,True,False,False,False,False,False,True,False
3,1408.0,2019,1,4,4,4,1,0,4,1403.858065,...,False,True,True,False,False,False,False,False,True,False
4,1250.0,2019,1,5,5,5,1,1,4,1403.858065,...,False,True,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518,2657.0,2023,2,27,0,58,9,0,4,1700.198582,...,False,True,True,False,False,False,False,False,True,False
1519,3922.0,2023,2,28,1,59,9,0,4,1700.198582,...,False,True,True,False,False,False,False,False,True,False
1520,3397.0,2023,3,1,2,60,9,0,1,3215.417323,...,False,True,True,False,False,False,False,False,True,False
1521,3195.0,2023,3,2,3,61,9,0,1,3215.417323,...,False,True,True,False,False,False,False,False,True,False


In [213]:
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import numpy as np

SEED = 42  # You can set your desired random seed

def generate_meta_features(train_x, train_y):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

    # Base models
    rf_model = RandomForestRegressor(random_state=SEED)
    xgb_model = XGBRegressor(random_state=SEED)
    catboost_model = CatBoostRegressor(random_state=SEED, verbose=False)
    lgbm_model = LGBMRegressor(random_state=SEED)

    # Generate predictions for base models using cross-validation
    rf_preds = cross_val_predict(rf_model, train_x, train_y, cv=cv, n_jobs=-1)
    xgb_preds = cross_val_predict(xgb_model, train_x, train_y, cv=cv, n_jobs=-1)
    catboost_preds = cross_val_predict(catboost_model, train_x, train_y, cv=cv, n_jobs=-1)
    lgbm_preds = cross_val_predict(lgbm_model, train_x, train_y, cv=cv, n_jobs=-1)

    # Prepare meta-features for the meta-learner
    meta_features = np.column_stack((rf_preds, xgb_preds, catboost_preds, lgbm_preds))

    return meta_features

def stacking_validation(train_ft):
    train_x = train_ft.drop(columns = 'price')
    train_y = train_ft['price']
    
    scaler = MinMaxScaler()
    train_x = pd.DataFrame(scaler.fit_transform(train_x))
    
    # Get meta-features from base models
    meta_features = generate_meta_features(train_x, train_y)

    # Stacking ensemble model (using LGBM as meta-learner)
    meta_model = LGBMRegressor(random_state=SEED)

    # Train the meta-learner on the meta-features
    rmse_scores = cross_val_score(meta_model, meta_features, train_y, cv=5, scoring=scorer, n_jobs=-1)
    rmse_score = np.mean(np.abs(rmse_scores))

    return rmse_score

def stacking_prediction(train_ft, test_ft):
    train_x = train_ft.drop(columns = 'price')
    train_y = train_ft['price']
    test_x = test_ft
    
    train_x = pd.DataFrame(scaler.fit_transform(train_x))
    test_x = pd.DataFrame(scaler.transform(test_x))

    # Get meta-features from base models
    meta_features = generate_meta_features(train_x, train_y)

    # Stacking ensemble model (using LGBM as meta-learner)
    meta_model = LGBMRegressor(random_state=SEED)

    # Train the meta-learner on the meta-features
    meta_model.fit(meta_features, train_y)

    # Generate predictions for test data using base models
    rf_model = RandomForestRegressor(random_state=SEED)
    xgb_model = XGBRegressor(random_state=SEED)
    catboost_model = CatBoostRegressor(random_state=SEED, verbose=False)
    lgbm_model = LGBMRegressor(random_state=SEED)

    rf_model.fit(train_x, train_y)
    xgb_model.fit(train_x, train_y)
    catboost_model.fit(train_x, train_y)
    lgbm_model.fit(train_x, train_y)


    rf_test_preds = rf_model.predict(test_x)
    xgb_test_preds = xgb_model.predict(test_x)
    catboost_test_preds = catboost_model.predict(test_x)
    lgbm_test_preds = lgbm_model.predict(test_x)

    # Prepare meta-features for test data
    test_meta_features = np.column_stack((rf_test_preds, xgb_test_preds, catboost_test_preds, lgbm_test_preds))

    # Generate final predictions using the meta-learner
    final_predictions = meta_model.predict(test_meta_features)

    return final_predictions


In [211]:
scores = []
for i in tqdm(range(39)):
    result = stacking_validation(globals()[f'train_ft_{i}'])
    scores.append(result)
np.mean(scores)

  0%|          | 0/39 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklea

603.9479109600088

In [214]:
# 예측 결과를 저장할 리스트
predictions_list = []

# 0부터 38까지의 train_ft_i와 test_ft_i에 대해 lgbm_prediction 함수 호출하고 결과를 리스트에 저장
for i in tqdm(range(39)):
    train_data = globals()[f'train_ft_{i}']
    test_data = globals()[f'test_ft_{i}']
    predictions = stacking_prediction(train_data, test_data)
    predictions_list.append(predictions)

# 리스트에 저장된 예측 결과를 하나의 NumPy 배열로 합치기
final_predictions = np.concatenate(predictions_list)

  0%|          | 0/39 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklea

In [217]:
submission['answer'] = final_predictions
submission
submission.to_csv('tmp_submission_name_1107.csv',index=False)

# 품목 (item) 별 

# 